<a href="https://colab.research.google.com/github/gahongayirebelise/NLP-FELLOWSHIP/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
%%bash

nvidia-smi

Fri Dec  9 10:54:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |   4072MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [83]:

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [84]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-mekd555x/farm-haystack_148cf58df63245f1b0758a0e16a34de7
  Resolved https://github.com/deepset-ai/haystack.git to commit 77cea8b1408699f69dc09420f9f9af3cf5e6f47a
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-mekd555x/farm-haystack_148cf58df63245f1b0758a0e16a34de7


In [85]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [86]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [87]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [88]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http


# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "/content/gdrive/MyDrive/NLP FELLOWSHIP/National dvlpt/data"
# s3_url = "https://www.minecofin.gov.rw/index.php?eID=dumpFile&t=f&f=12361&token=56d0948f7b3b2df6f66ad4e9d6041164770b8ce9"
# fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>", "content": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO:haystack.utils.preprocessing:Converting /content/gdrive/MyDrive/NLP FELLOWSHIP/National dvlpt/data/National_Investment_Policy.txt
INFO:haystack.utils.preprocessing:Converting /content/gdrive/MyDrive/NLP FELLOWSHIP/National dvlpt/data/Vision_2020_.txt


[<Document: {'content': 'entry  and  scrutiny  for  all  central  government  projects  Experience  has  shown  that  this  measure has considerably contributed to the improvement of the quality and transparency  of  public  projects  Significant  progress  has  also  been  achieved  in  other  areas  of  public  investments,  which  now  call  for  further  policy  guidance:  eg  development  of  a  PPP  framework  and  the  introduction  of  PPPs,  alternative  forms  of  investment  engaging  the  private sector, advancements in decentralization   The  objective  of  this  policy  is  to  achieve  the  country’s  strategic  development  goals  by  transforming  the  “National  Public  Investment  Policy”  into  a  “National  Investment  Policy”,  which  addresses  public  as  well  as  private  investment  It  is  intended  to  lay  the  ground  to  carefully  balance  new  public  investment  projects  and  potential  dis-investment  needs  with  options  to  strengthen  private  s

In [89]:
# An in-memory TfidfRetriever based on Pandas dataframes
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.nodes.retriever.sparse:Found 2 candidate paragraphs from 2 docs in DB


In [90]:
from haystack.nodes import FARMReader

#Download and initiate the model
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [91]:
#Bundle together the model and retriver(Which is the extractor)
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [171]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="what are The overarching goals for the Vision 2050", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/5 [00:00<?, ? Batches/s]

ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-23139, -23076) with a span answer. 


In [172]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'strategic  development  goals', 'type': 'extractive', 'score': 0.5757510662078857, 'context': 'bjective  of  this  policy  is  to  achieve  the  country’s  strategic  development  goals  by  transforming  the  “National  Public  Investment  Poli', 'offsets_in_document': [{'start': 599, 'end': 628}], 'offsets_in_context': [{'start': 61, 'end': 90}], 'document_id': '45cfe797186bb10c3fdd541348a0666f', 'meta': {'name': 'National_Investment_Policy.txt'}}>,
             <Answer {'answer': 'knowledge  transfer  and  promotion  of  the  local  economy', 'type': 'extractive', 'score': 0.5371639728546143, 'context': 'ction  shall  consider  the  objectives  of  knowledge  transfer  and  promotion  of  the  local  economy,  especially  targeting the creation of loca', 'offsets_in_document': [{'start': 14756, 'end': 14816}], 'offsets_in_context': [{'start': 45, 'end': 105}], 'document_id': '45cfe797186bb10c3fdd541348a0666f', 'meta': {'name': 'National_Investment_Po

In [ ]:
# ...or use a util to simplify the output
from haystack.utils import print_answers


# Change `minimum` to `medium` or `all` to control the level of detail
print_answers(prediction, details="minimum")

In [170]:
def get_answer(q):
  prediction = pipe.run(
      query="The overarching goals for the Vision 2050", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 1}}
  )
  return prediction

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19917 sha256=ab6a75b6c58935cd80df95636813bad04a3d5db335d50e0aadd381f8114248be
  Stored in directory: /root/.cache/pip/wheels/b9/a5/88/18b50a429b7c2543af4837217f05810da8c7ca7964fa9d98f2
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3133875 sha256=02007e17efda13ec7e3bef086045ea6002bbb0e1f98e64de267804b9e4daeb3d
  Stored in directory: /root/.cache/pip/wheels/b8/6e/68/d9e7236903651c82c9ad4e2aca0c8fb462c7569bf619ebc67a
Successfully built easynmt 

In [127]:
from easynmt import EasyNMT

model = EasyNMT('OPUS-MT')

question = model.translate(raw_query, source_lang=lang, target_lang='en')

answer = model.translate(answer, source_lang='en', target_lang=lang)

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [54]:
print(answer)

Pas de réponse


In [32]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [151]:
from langdetect import detect

raw_query = "DE PIJLERS VAN VISIE 2020"
lang = detect(raw_query)
print(lang)

vi


In [129]:
question

'President of the Republic of Rwanda'